#1.安装包

In [ ]:
!pip install langchain
!pip install openai
!pip install chromadb


#2.准备API KEY

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import openai

import os

os.environ["OPENAI_API_KEY"] = "[you api key]"


#3.嵌入数据到向量数据库
文本切割：
在处理文本数据时，将文本分割成更小的片段（例如单词、短语或句子）是一种常见的做法。这是因为语言模型通常在这些更小的文本片段上进行训练，而不是在整个文档或长段落上。这样做有几个原因：

计算效率：处理更小的文本片段通常比处理整个文档更快，因为模型需要处理的数据量更小。

上下文理解：在许多情况下，一个单词或短语的含义取决于其周围的上下文。通过在更小的文本片段上进行训练，模型可以更好地理解这些上下文。

内存管理：大型语言模型（如GPT-3或BERT）通常有一个固定的输入长度限制（例如，GPT-3的最大输入长度为2048个标记）。将文本分割成更小的片段可以确保模型能够处理所有的输入，而不会因为输入过长而无法处理。

metadatas：元数据是用来标识每个文本片段的来源的。例如，"0-pl"可能表示第一个文本片段的来源，"1-pl"表示第二个文本片段的来源，以此类推。


In [5]:
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

data="xiao ming likes blue，he is from China"
text_splitter = CharacterTextSplitter()
texts = text_splitter.split_text(data)
print(texts)

#选择嵌入模型
embeddings = OpenAIEmbeddings(deployment='text-embedding-ada-002')

persist_directory = 'db'
#将文本保存到Chroma 向量数据库中
docsearch = Chroma.from_texts(
    texts,
    embeddings,
    persist_directory=persist_directory,
    metadatas=[{"source":f"{i}-pl"} for i in range(len(texts))]
)


['xiao ming likes blue，he is from China']


APITimeoutError: Request timed out.

#4.测试向量数据库

chain_type的值"stuff"指的是在Langchain库中使用的链的类型。"stuff"链，也被称为StuffDocumentsChain，是一种文档链，它接收一系列文档，将它们全部插入到一个提示中，然后将该提示传递给LLM（语言模型）。这种链适用于文档较小且大部分调用只传入少量文档的应用。这是一种直接的文档摘要方法。

In [18]:
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain

llm = OpenAI()
#转换为一个检索器，进行后续的查找
retriever=docsearch.as_retriever()
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm = llm,
    #Langchain的一种文档链，接受文档
    chain_type="stuff",
    retriever=retriever,
    return_source_documents = True
)

def process_result(result):
  print(result['answer'])

question = '小明喜欢什么颜色？'
#将字典question 作为参数传入，并获取answer作为结果
result = chain({"question":question})
process_result(result)


 Xiao Ming likes blue.

